In [ ]:
from dash import Dash, html, dcc, Input, Output, callback, State
import pandas as pd
import plotly.express as px
import os
import matplotlib.pyplot as plt
import base64
from io import BytesIO

import xarray as xr
import numpy as np
import pandas as pd
import netCDF4

import oggm
from oggm import cfg, utils, workflow, tasks, graphics, global_tasks, DEFAULT_BASE_URL
from oggm.core import massbalance, flowline
from oggm.sandbox import distribute_2d

from oggm.tasks import process_w5e5_data
from oggm.tasks import process_cru_data
from oggm.tasks import process_histalp_data
from oggm.tasks import process_ecmwf_data
from oggm.shop.ecmwf import get_ecmwf_file

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

# Function to initialize glacier directories
def init_directories(gval):
    cfg.initialize()
    cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-GettingStarted-10m')
    rgi_ids = [gval]
    gdirs = workflow.init_glacier_directories(
        rgi_ids,
        prepro_base_url=DEFAULT_BASE_URL,
        from_prepro_level=4,
        prepro_border=80
    )
    return gdirs
app.title = 'RGI/OGGM Visualizer'
app.layout = html.Div([
    html.Div([
    html.H1(children='RGI/OGGM Visualizer', style={'textAlign': 'center'}),
        html.Div([
                dcc.Input(id="id-selection", type="text", value="RGI60-11.00897", placeholder="RGIId", style={'marginRight': '10px'}),
                html.Button(id='submit-button-state', n_clicks=0, children='Load Glacier', style={'marginTop': '10px'}),
            dcc.Dropdown(
                options=[
                {'label': 'ELA Change', 'value': 'ela'},
                {'label': 'MB Change', 'value': 'mb'},
                {'label': 'Flowline', 'value': 'flowline'}  
                ],
                value='ela',
                id='plot-selection',
                multi=False,
                placeholder="Select a variable",
                style={'marginTop': '5px'}
            ),
            # dcc.RadioItems(
            #     ['Linear', 'Log'],
            #     'Linear',
            #     id='crossfilter-xaxis-type',
            #     labelStyle={'display': 'none', 'marginTop': '10px'}
            # )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='interactive-graph',
            style={'height': '100%', 'width': '100%', 'display': 'none'},
            # hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'},
    id='graph-container'),


    html.Div([
        # html.H2(children='Glacier Domain', style={'display': 'none'}, id='domain-title'),
        html.Img(id='glacier-domain', style ={'width': 'auto', 'height': 'auto', 'max-width': '100%', 'max-height': '100%'}),
    ], style={'display': 'inline-block', 'width': '49%', 'textAlign': 'center', 'padding': '0 20'}),

    html.Div(dcc.Slider(
        2000,
        2019,
        step=None,
        id='crossfilter-year--slider',
        value=2019,
        marks={str(year): str(year) for year in range(2000, 2020)},
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px', 'display': 'none'}, id='slider-container'),
])


@callback(
    Output('glacier-domain', 'src'),
    Output('slider-container', 'style'),
    # Output('crossfilter-xaxis-type', 'labelStyle'),
    Output('interactive-graph', 'figure'),
    Output('interactive-graph', 'style'),
    Input('submit-button-state', 'n_clicks'),
    Input('plot-selection', 'value'),
    Input('crossfilter-year--slider', 'value'),
    State('id-selection', 'value'),
    prevent_initial_call = True
    # Input('crossfilter-indicator-scatter', 'hoverData'),
    # Input('crossfilter-yaxis-column', 'value'),
    # Input('crossfilter-yaxis-type', 'value')
    )

def update_glacier_domain(n_clicks, plot_type, year_value, value):
    if n_clicks == 0:
        # Do not display the graph or domain until the glacier is loaded
        return '', {'display': 'none'}, {}, {'display': 'none'}

    elif n_clicks > 0:
        try:
            gdirs = init_directories(value)
            #Graph 1 - ELA
            gdir = gdirs[0]
            fig = plt.figure()
            ax = fig.add_subplot(111)
            graphics.plot_domain(gdir, ax=ax)
            ax.set_title('Glacier Map for ' + gdir.rgi_id)
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            plt.tight_layout()

            buf = BytesIO()
            fig.savefig(buf, format="png")
            fig_data = base64.b64encode(buf.getbuffer()).decode("ascii")

            if plot_type == 'ela':
                # ELA Change
                global_tasks.compile_ela(gdirs, ys=2000, ye=year_value)
                ela_df = pd.read_hdf(os.path.join(cfg.PATHS['working_dir'], 'ELA.hdf'))
                fig = px.scatter(
                    ela_df,
                    labels={"index": "Year", "value": "ELA (m)"},
                    title=f"Equilibrium Line Altitude (ELA) Change for {gdir.rgi_id}",
                    trendline="ols",
                    trendline_color_override="grey",
                )
                fig.update_layout(
                    xaxis_title="Year",
                    yaxis_title="ELA (m)",
                    template="plotly_white",
                    showlegend=False,
                    legend_title_text="",
                    legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.02,
                        xanchor="right",
                        x=1
                    )
                    

                )

            elif plot_type == 'mb':
                # MB Change
                mbmod = massbalance.MultipleFlowlineMassBalance(gdir)
                fls = gdir.read_pickle('model_flowlines')
                years = np.arange(1950, year_value + 1)
                mb_ts = mbmod.get_specific_mb(fls=fls, year=years)
                fig = px.scatter(
                    x=years, 
                    y=mb_ts, 
                    labels={"x": "Year", "y": "Specific MB (m w.e.)"},
                    title=f"Mass Balance (MB) Change for {gdir.rgi_id}",
                    trendline="ols",
                    trendline_color_override="grey",
                )
                fig.update_layout(
                    xaxis_title="Year",
                    yaxis_title="Specific MB (m w.e.)",
                    template="plotly_white"
                )
            elif plot_type == 'flowline':
                
                workflow.execute_entity_task(tasks.compute_inversion_velocities, gdirs)

                inversion_output = gdir.read_pickle('inversion_output')  # The task above wrote the data to a pickle file - but we write to plenty of other files!

                # Take the first flowline
                fl = inversion_output[0]
                # the last grid points often have irrealistic velocities
                # because of the equilibrium assumption
                vel = fl['u_surface'][:-1]
                flowlinevar = [i for i in range(1,len(vel)+1)]
                fig = px.line(
                    x=flowlinevar, 
                    y=vel, 
                    labels={"x": "Flowline", "y": "velocity (m yr-1)"},
                    title=f"Flowline Velocity for {gdir.rgi_id}"
                )
                fig.update_layout(
                    xaxis_title="Flowline",
                    yaxis_title="Velocity",
                    template="plotly_white",
                    # showxticklabels=False,
                )
                
                
                  

            return [f'data:image/png;base64,{fig_data}', {'width': '49%', 'padding': '0px 20px 20px 20px', 'display': 'block'}, fig, {'display': 'block'}]
    
        except Exception as e:
            #print(f"Error: {e}")
            return ([], {'display': 'none'}, [], {'display': 'none'})
    else:
        return ['']


if __name__ == '__main__':
    app.run(debug=True, port=8050)


2025-04-21 11:12:49: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2025-04-21 11:12:49: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2025-04-21 11:12:49: oggm.cfg: Multiprocessing: using all available processors (N=8)
2025-04-21 11:12:50: oggm.workflow: init_glacier_directories from prepro level 4 on 1 glaciers.
2025-04-21 11:12:50: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 1 glaciers
2025-04-21 11:12:50: oggm.utils: Applying global task compile_ela on 1 glaciers
2025-04-21 11:12:50: oggm.workflow: Execute entity tasks [compute_ela] on 1 glaciers
2025-04-21 11:12:50: oggm.core.massbalance: (RGI60-11.00897) compute_ela
